In [74]:
import numpy as np

In [75]:
# trace ray with no reflection 
# different r should give the same value as long as the sensor plane is at the focus point for the lens
# different theta should change it linearly

# ideas: keep this part in python and integrate w/ cpp?

def T(d):
    return np.matrix([[1, d], [0, 1]])
    
def R(ri, n1, n2):
    return np.matrix([[1, 0], [(n1-n2)/(n2*ri), n1/n2]])
    
def L(ri):
    return np.matrix([[1, 0], [2/ri, 1]])

pupil_height=12.3

R1 = R(30.810, 1.000, 1.652)
T1 = T(7.700)
L1 = L(1.652)

R2 = R(-89.350, 1.652, 1.603)
T2 = T(1.850)
L2 = L(1.603)

R3 = R(580.380, 1.603, 1.000)
T3 = T(3.520)
L3 = L(1)

R4 = R(-80.630, 1.000, 1.643)
T4 = T(1.850)
L4 = L(1.643)

R5 = R(28.340, 1.643, 1.000)
T5 = T(4.180)
L5 = L(1)

# now, we've reached the aperature plane
# height = 11.6
T6 = T(3.000)

R7 = np.matrix([[1, 0], [0, 1.000/1.581]])
T7 = T(1.850)
L7 = L(1.581)

R8 = R(32.190, 1.581, 1.694)
T8 = T(7.270)
L8 = L(1.694)

R9 = R(-52.990, 1.694, 1)
T9 = T(81.320)
L9 = L(1)

def trace_ray(r, theta):
    r = np.array([r, theta])

    Ma = T5.dot(R5).dot(T4).dot(R4).dot(T3).dot(R3).dot(T2).dot(R2).dot(T1).dot(R1)
    Ms = T9.dot(R9).dot(T8).dot(R8).dot(T7).dot(R7).dot(T6)
    
    return Ms.dot(Ma).dot(r)
    
def trace_ray_with_ghost(r, theta):
    r = np.array([r, theta])
    
    R3_inv = np.linalg.inv(R3)
    L2_inv = np.linalg.inv(L2)
 

    M1 = T3.dot(R3).dot(T2).dot(R2).dot(T1).dot(R1)
    Bounce = T2.dot(L2_inv).dot(T2).dot(R3_inv).dot(T3).dot(L4)
    M2 = T5.dot(R5).dot(T4).dot(R4).dot(T3).dot(R3)
    Ms = T9.dot(R9).dot(T8).dot(R8).dot(T7).dot(R7).dot(T6)
    
    return Ms.dot(M2).dot(Bounce).dot(M1).dot(r)

def trace_ray_with_ghost_2(r, theta):
    r = np.array([r, theta])
    
    R3_inv = np.linalg.inv(R3)
    L1_inv = np.linalg.inv(L1)
    R2_inv = np.linalg.inv(R2)
 

    M1 = T3.dot(R3).dot(T2).dot(R2).dot(T1).dot(R1)
    # changed t2 to t1
    Bounce = T1.dot(L1_inv).dot(T1).dot(R2_inv).dot(T2).dot(R3_inv).dot(T3).dot(L4)
    # 
    M2 = T5.dot(R5).dot(T4).dot(R4).dot(T3).dot(R3).dot(T2).dot(R2)
    Ms = T9.dot(R9).dot(T8).dot(R8).dot(T7).dot(R7).dot(T6)
    
    return Ms.dot(M2).dot(Bounce).dot(M1).dot(r)

In [76]:
# different ghosts should have different sizes
print(trace_ray_with_ghost(12.3, 0.001))
print(trace_ray_with_ghost_2(12.3, 0.001))
# changing theta
print(trace_ray_with_ghost_2(12.3, 0.01))
print(trace_ray_with_ghost_2(12.3, 0.02))
print(trace_ray_with_ghost_2(12.3, 0.03))

[[-10424.52563746    -99.65643518]]
[[-22351.43750574   -201.59049826]]
[[-22560.88216938   -203.47951165]]
[[-22793.5984623    -205.57841542]]
[[-23026.31475523   -207.67731919]]


In [77]:
# auto generations & 1-indexing
Rs = [None, R1, R2, R3, R4, R5, None, R7, R8, R9]
Ts = [None, T1, T2, T3, T4, T5, T6, T7, T8, T9]
Ls = [None, L1, L2, L3, L4, L5, None, L7, L8, L9]

def trace_ray_auto(r, theta, i, j):
    rv = np.array([r, theta]).reshape(1, -1).T
    
    # forward through j-1
    for k in range(1, j):
        # print(k, 'rv', rv, rv.shape)
        rv = Ts[k].dot(Rs[k]).dot(rv)
    
    # reflect off of Lj
    rv = Ls[j].dot(rv)
    
    # Inverse
    for k in range(i+1, j-2, -1):
        rv = np.linalg.inv(Rs[k]).dot(Ts[k]).dot(rv)
    
    rv = Ts[i].dot(np.linalg.inv(Ls[i])).dot(Ts[i]).dot(rv)
    
    for k in range(i+1, 10):
        if k == 6:
            # crossing the aperture!
            rv = Ts[k].dot(rv)
            continue
        rv = Ts[k].dot(Rs[k]).dot(rv)
    
    return rv

In [78]:
print(trace_ray_auto(12.3, 0.001, 2, 4))
print(trace_ray_with_ghost(12.3, 0.001))

print(trace_ray_auto(10.5, 0.03, 2, 4))
print(trace_ray_with_ghost(10.5, 0.03))

[[-10424.52563746]
 [   -99.65643518]]
[[-10424.52563746    -99.65643518]]
[[-9212.73479393]
 [  -88.07198507]]
[[-9212.73479393   -88.07198507]]


In [79]:
print(trace_ray_auto(12.3, 0.03, 1, 4))
print(trace_ray_with_ghost_2(12.3, 0.03))

[[-18368.33798444]
 [  -165.71932704]]
[[-23026.31475523   -207.67731919]]
